In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
files = glob('threads/*.json')
len(files)

10565

In [3]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [13]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    for d in data:
        if d[-1] > 1:
            url = d[0]
            for i in range(2, d[-1] + 1, 1):
                urls.append(f'{url}page-{i}')

100%|████████████████████████████████████| 10565/10565 [00:11<00:00, 953.16it/s]


In [18]:
urls = sorted(list(set(urls)))

In [19]:
len(urls)

767956

In [15]:
urls[0]

'https://forums.hardwarezone.com.sg/threads/if-accidentally-inhale-a-cylinder-of-lpg-will-up-lorry-not.6894690/page-2'

In [22]:
import time

def get_href_topic(url, topic = None):
    while True:
        try:
            r = requests.get(url, timeout = 10.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    div = soup.find_all('div', {'class': 'bbWrapper'})
    return [str(d) for d in div]

In [23]:
max_worker = 50
for i in tqdm(range(0, len(urls), max_worker)):
    
    filename = os.path.join('pages', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
    
    results = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_href_topic, url): url for url in urls[i: i + max_worker]}

        for future in as_completed(futures):
            results.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)
        
    try:
        del results
        del futures
    except:
        pass

 54%|██████████████████▎               | 8271/15360 [9:11:42<7:38:04,  3.88s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████████████████████████████| 15360/15360 [17:04:14<00:00,  4.00s/it]


In [24]:
files = glob('threads/*.json')
len(files)

10565

In [25]:
texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d[1]:
            texts.append(BeautifulSoup(d_, "lxml").text)

100%|█████████████████████████████████████| 10565/10565 [08:08<00:00, 21.62it/s]


In [26]:
files = glob('pages/*.json')
len(files)

15360

In [27]:
len(texts)

3059618

In [28]:
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d:
            texts.append(BeautifulSoup(d_, "lxml").text)

 68%|█████████████████████████▎           | 10510/15360 [33:44<15:17,  5.28it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
len(texts)

17525287

In [30]:
texts = list(set(texts))
len(texts)

16698134

In [31]:
with open('everything.jsonl', 'w') as fopen:
    for t in tqdm(texts):
        if len(t):
            fopen.write(f'{json.dumps(t)}\n')

100%|███████████████████████████| 16698134/16698134 [00:23<00:00, 707383.91it/s]


In [2]:
# !zip -r pages.zip pages
# !zip -r threads.zip threads
# !zip -r topics.zip topics